<a href="https://colab.research.google.com/github/cshivak/AllureRepo/blob/master/VideoSummarization_PresentationNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:
https://towardsdatascience.com/object-detection-and-tracking-in-pytorch-b3cf1a696a98

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [0]:
!git clone https://github.com/cfotache/pytorch_objectdetecttrack.git

In [0]:
!pip install ffmpeg-python
!pip install filterpy

In [0]:
import sys
sys.path.append('/content/pytorch_objectdetecttrack')

In [0]:
from models import *
from utils import *
import ffmpeg

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import pandas as pd
from google.colab.patches import cv2_imshow
import imutils
import cv2
from IPython.display import clear_output
from collections import defaultdict
import numpy as np
import math
import shutil
import progressbar

In [0]:
### Background subtraction option1
### Color images with subtraction written as a video
input_video = '/content/1.mp4'
output_video = '/content/video_bs.mp4'
probe = ffmpeg.probe(input_video)
video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
duration = int(math.ceil(float(video_stream['duration'])))
width = int(video_stream['width'])
height = int(video_stream['height'])
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video=cv2.VideoWriter(output_video, fourcc,10,(width,height))

cap = cv2.VideoCapture(input_video)
# fgbg = cv2.createBackgroundSubtractorMOG2()
fgbg = cv2.createBackgroundSubtractorKNN()

try:
  while(1):
      ret, frame = cap.read()

      fgmask = fgbg.apply(frame)
      mask_rgb = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2RGB)
      out_frame = cv2.bitwise_and(frame, mask_rgb)
      video.write(out_frame)
      k = cv2.waitKey(30)
      if k == 27:
          break

  cap.release()
  video.release()
  cv2.destroyAllWindows()
except:
  cap.release()
  video.release()
  cv2.destroyAllWindows()

In [0]:
### Background subtraction option-2
# inputVideo = '/content/1.mp4'
# output_video = '/content/video_bs.mp4'
# cap = cv2.VideoCapture(inputVideo)
# probe = ffmpeg.probe(inputVideo)
# video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
# duration = int(math.ceil(float(video_stream['duration'])))
# width = int(video_stream['width'])
# height = int(video_stream['height'])
# fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
# video=cv2.VideoWriter(output_video, fourcc,10,(width,height))
# #fgbg = cv2.createBackgroundSubtractorMOG2()
# fgbg = cv2.createBackgroundSubtractorKNN(detectShadows = False)
# kernel = np.ones((2,2),np.uint8)
# i = 0
# try:
#   while True:
#       ret, frame = cap.read()
#       # cv2_imshow('orginal',frame)
#       frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#       frame_hsv[:, :, 2] = cv2.equalizeHist(frame_hsv[:, :, 2])
#       frame = cv2.cvtColor(frame_hsv, cv2.COLOR_HSV2BGR)
#       fgmask = fgbg.apply(frame)
#       fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
#       fgmask = cv2.blur(fgmask,(3,3))
#       # cv2.imshow("Frame",fgmask)
#       red_frame = cv2.bitwise_and(frame[:,:,2], fgmask)
#       green_frame = cv2.bitwise_and(frame[:,:,1], fgmask)
#       blue_frame = cv2.bitwise_and(frame[:,:,0], fgmask)
#       # cv2_imshow(cv2.merge((blue_frame,green_frame,red_frame)))
#       video.write(cv2.merge((blue_frame,green_frame,red_frame)))
#       clear_output(wait=True)
#       k = cv2.waitKey(30)
#       if k == 27:
#           break
#   cv2.destroyAllWindows()
#   cap.release()
#   video.release()
# except:
#   cv2.destroyAllWindows()
#   cap.release()
#   video.release()

In [0]:
### YOLO needs this. May go away if different implementation is used
config_path='/content/pytorch_objectdetecttrack/config/yolov3.cfg'
weights_path='/content/yolov3.weights'
class_path='/content/pytorch_objectdetecttrack/config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4
# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

In [0]:
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms=transforms.Compose([transforms.Resize((imh,imw)),
         transforms.Pad((max(int((imh-imw)/2),0), 
              max(int((imw-imh)/2),0), max(int((imh-imw)/2),0),
              max(int((imw-imh)/2),0)), (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, 
                        conf_thres, nms_thres)
    return detections[0]

In [0]:
### May go away if different implementation is used completely or partially
## Use background subtracted video as input and write frames
videopath = os.path.join(os.getcwd(), 'video_bs.mp4')
cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]
# initialize Sort object and video capture
from sort import *
vid = cv2.VideoCapture(videopath)
mot_tracker = Sort()
# dict = defaultdict(list) 
# objectBBox = defaultdict(list) 
summaryDf = pd.DataFrame(columns=['objectId', 'frame', 'x', 'y', 'x+w', 'y+w'])

probe = ffmpeg.probe(videopath)
video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
duration = int(math.ceil(float(video_stream['duration'])))
width = int(video_stream['width'])
height = int(video_stream['height'])
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
# video=cv2.VideoWriter(os.path.join(os.getcwd(), 'video.mp4'),fourcc,10,(width,height))
if os.path.exists('/content/Images'):
  shutil.rmtree('/content/Images')
  os.mkdir('/content/Images')
else: 
  os.mkdir('/content/Images')
#while(True):
for ii in range(duration):
    ret, frame = vid.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pilimg = Image.fromarray(frame)
    detections = detect_image(pilimg)
    img = np.array(pilimg)
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x
    if detections is not None:
        tracked_objects = mot_tracker.update(detections.cpu())
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
            box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
            y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
            x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])
            color = colors[int(obj_id) % len(colors)]
            color = [i * 255 for i in color]
            cls = classes[int(cls_pred)]
            # cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h),
            #              color, 1)
            # cv2.rectangle(frame, (x1, y1-35), (x1+len(cls)*19+60,
            #              y1), color, -1)
            # cv2.putText(frame, str(obj_id), 
            #             (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 
            #             1, (255,255,255), 1)
            data = [{'objectId':obj_id,'frame':ii,'x':x1,'y':y1,'x+w': x1+box_w, 'y+w': y1+box_h}]
            summaryDf = summaryDf.append(data)
            # dict[obj_id].append(ii)
            # objectBBox[obj_id].append((x1, y1, x1+box_w, y1+box_h))
            # print(obj_id, ii)
    # fig=figure(figsize=(12, 8))
    # title("Video Stream")
    # imshow(frame)
    # show()
    cv2.imwrite(os.path.join('/content/Images', '{0}.png'.format(ii)),frame)
    # video.write(frame)
    clear_output(wait=True)
    
# file.close()

In [0]:
### summaryDf now has info on which frame an object appears and it's coordinates
summaryDf['frame'].sort_values().unique()

In [0]:
summaryDf['objectId'].unique()

In [0]:
summaryDf.shape

In [0]:
def pasteImageBlackBg(s_img, x_offset, y_offset, imgId, frame):
  # Use regex to get text
  # text = imgPath.split('/')[-1].split('.p')[0]
  # s_img = cv2.imread(imgPath, -1)
  l_img = np.zeros((500,634,3), np.uint8)

  y1, y2 = y_offset, y_offset + s_img.shape[0]
  x1, x2 = x_offset, x_offset + s_img.shape[1]

  alpha_s = s_img[:, :, 2] / 255.0
  alpha_l = 1.0 - alpha_s

  for c in range(0, 3):
      l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] +
                                alpha_l * l_img[y1:y2, x1:x2, c])
      
  cv2.putText(l_img, '{0}-{1}'.format(imgId, frame), 
              (x_offset + 40, y_offset-10), cv2.FONT_HERSHEY_SIMPLEX, 
              0.30, (255,255,255), 1)
  return l_img

In [0]:
### Crop images using bounding boxes and extract person based on objectId and paste on black background
imagesPath = '/content/Images'
cropImagePath = '/content/ImagesCropped'

if os.path.exists(cropImagePath):
  shutil.rmtree(cropImagePath)
  os.mkdir(cropImagePath)
else:
  os.mkdir(cropImagePath)
for object in list(summaryDf['objectId']):
  for frame in summaryDf[summaryDf['objectId'] == object]['frame']:
    if(frame > 10):
      row = summaryDf[(summaryDf['objectId'] == object) & (summaryDf['frame'] == frame)]
      x = int(row['x'])
      y = int(row['y'])
      yw = int(row['y+w'])
      xw = int(row['x+w'])
      img = cv2.imread(os.path.join(imagesPath, '{0}.png'.format(frame)))
      cropImg = img[y:yw, x:xw]
      pastImg = pasteImageBlackBg(cropImg, x, y, object, frame)
      cv2.imwrite(os.path.join(cropImagePath, '{0}_{1}.png'.format(object, frame)),pastImg)

In [0]:
# Frames under 30 are just blank. Maynot need this. Investigate??
summaryDf = summaryDf[summaryDf['frame'] > 39]
summaryDf.head()

In [0]:
croppedFolderPath = '/content/ImagesCropped'
overLaidImagePath = '/content/OverlaidImages'
output_video = '/content/final.mp4'
if os.path.exists(overLaidImagePath):
  shutil.rmtree(overLaidImagePath)
  os.mkdir(overLaidImagePath)
else:
  os.mkdir(overLaidImagePath)
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video=cv2.VideoWriter(output_video, fourcc,10,(634,500))
i = 0
while summaryDf.shape[0]!= 0:
  x = summaryDf.groupby(summaryDf['objectId']).first()
  x['objectId'] = x.index
  x['fileNames'] = x.apply(lambda row: '{0}_{1}.png'.format(row['objectId'], int(row['frame'])), axis=1)
  l_img = np.zeros((500,634,3), np.uint8)
  for fileName in list(x['fileNames']):
    # print(os.path.join(croppedFolderPath ,fileName))
    img = cv2.imread(os.path.join(croppedFolderPath ,fileName))
    # cv2_imshow(img)
    l_img = cv2.addWeighted(l_img,1.0,img,2.0,0)
  # cv2_imshow(l_img)
  cv2.imwrite(os.path.join(overLaidImagePath, '{0}.png'.format(i)),l_img)
  video.write(l_img)
  x.drop(['fileNames'], axis = 1, inplace = True )
  summaryDf = pd.concat([summaryDf, x])
  summaryDf = summaryDf.drop_duplicates(keep=False)
  i += 1

cv2.destroyAllWindows()
video.release()

In [0]:
output_video = '/content/final.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video=cv2.VideoWriter(output_video, fourcc,10,(634,500))
for i in range(0, 57):
  img = cv2.imread(os.path.join(overLaidImagePath, '{0}.png'.format(str(i))))
  # print(os.path.join(overLaidImagePath, '{0}.png'.format(str(i))))
  # cv2_imshow(img)
  video.write(img)

cv2.destroyAllWindows()
video.release()
